In [1]:
class Stock:
    def __init__(self, symbol, date_, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date_
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol, 
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
    def as_dict(self):
        return dict(
            symbol=self.symbol,
            timestamp=self.timestamp,
            order=self.order,
            price=self.price,
            volume=self.volume,
            commission=self.commission)

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [3]:
from marshmallow import Schema,fields

In [6]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal()
    high = fields.Decimal()
    low = fields.Decimal()
    close = fields.Decimal()
    volume = fields.Integer()
    

In [7]:
StockSchema().dump(Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

{'high': Decimal('338.64'),
 'date': '2018-11-22',
 'close': Decimal('338.19'),
 'low': Decimal('337.60'),
 'open': Decimal('338.19'),
 'volume': 365607,
 'symbol': 'TSLA'}

In [8]:
StockSchema().dumps(Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

TypeError: Object of type 'Decimal' is not JSON serializable

In [10]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string = True)
    high = fields.Decimal(as_string = True)
    low = fields.Decimal(as_string = True)
    close = fields.Decimal(as_string = True)
    volume = fields.Integer()

In [11]:
StockSchema().dump(Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

{'high': '338.64',
 'date': '2018-11-22',
 'close': '338.19',
 'low': '337.60',
 'open': '338.19',
 'volume': 365607,
 'symbol': 'TSLA'}

In [12]:
class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string = True)
    commission = fields.Decimal(as_string = True)
    volume = fields.Integer()

In [14]:
TradeSchema().dumps(Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')))

'{"price": "338.25", "order": "buy", "volume": 100, "timestamp": "2018-11-22T10:05:12", "commission": "9.99", "symbol": "TSLA"}'

In [15]:
class ActivitySchema(Schema):
    quotes = fields.Nested(StockSchema,many = True)
    trades = fields.Nested(TradeSchema,many = True)   

In [21]:
result = ActivitySchema().dumps(activity,indent=2)

In [22]:
type(result)

str

In [23]:
print(result)

{
  "quotes": [
    {
      "high": "338.64",
      "date": "2018-11-22",
      "close": "338.19",
      "low": "337.60",
      "open": "338.19",
      "volume": 365607,
      "symbol": "TSLA"
    },
    {
      "high": "177.25",
      "date": "2018-11-22",
      "close": "176.78",
      "low": "176.64",
      "open": "176.66",
      "volume": 3699184,
      "symbol": "AAPL"
    },
    {
      "high": "103.48",
      "date": "2018-11-22",
      "close": "103.11",
      "low": "103.07",
      "open": "103.25",
      "volume": 4493689,
      "symbol": "MSFT"
    }
  ],
  "trades": [
    {
      "price": "338.25",
      "order": "buy",
      "volume": 100,
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "symbol": "TSLA"
    },
    {
      "price": "177.01",
      "order": "sell",
      "volume": 20,
      "timestamp": "2018-11-22T10:30:05",
      "commission": "9.99",
      "symbol": "AAPL"
    }
  ]
}


In [24]:
activity_dser = ActivitySchema().loads(result)
type(activity_dser)

dict

In [25]:
print(activity_dser)

{'quotes': [{'high': Decimal('338.64'), 'date': datetime.date(2018, 11, 22), 'close': Decimal('338.19'), 'low': Decimal('337.60'), 'open': Decimal('338.19'), 'volume': 365607, 'symbol': 'TSLA'}, {'high': Decimal('177.25'), 'date': datetime.date(2018, 11, 22), 'close': Decimal('176.78'), 'low': Decimal('176.64'), 'open': Decimal('176.66'), 'volume': 3699184, 'symbol': 'AAPL'}, {'high': Decimal('103.48'), 'date': datetime.date(2018, 11, 22), 'close': Decimal('103.11'), 'low': Decimal('103.07'), 'open': Decimal('103.25'), 'volume': 4493689, 'symbol': 'MSFT'}], 'trades': [{'price': Decimal('338.25'), 'order': 'buy', 'volume': 100, 'timestamp': datetime.datetime(2018, 11, 22, 10, 5, 12), 'commission': Decimal('9.99'), 'symbol': 'TSLA'}, {'price': Decimal('177.01'), 'order': 'sell', 'volume': 20, 'timestamp': datetime.datetime(2018, 11, 22, 10, 30, 5), 'commission': Decimal('9.99'), 'symbol': 'AAPL'}]}


In [26]:
from pprint import pprint

In [27]:
pprint(activity_dser)

{'quotes': [{'close': Decimal('338.19'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('338.64'),
             'low': Decimal('337.60'),
             'open': Decimal('338.19'),
             'symbol': 'TSLA',
             'volume': 365607},
            {'close': Decimal('176.78'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('177.25'),
             'low': Decimal('176.64'),
             'open': Decimal('176.66'),
             'symbol': 'AAPL',
             'volume': 3699184},
            {'close': Decimal('103.11'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('103.48'),
             'low': Decimal('103.07'),
             'open': Decimal('103.25'),
             'symbol': 'MSFT',
             'volume': 4493689}],
 'trades': [{'commission': Decimal('9.99'),
             'order': 'buy',
             'price': Decimal('338.25'),
             'symbol': 'TSLA',
             'timestamp': datetim

In [44]:
from marshmallow import post_load

class TradeSchema(Schema):
    
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string = True)
    commission = fields.Decimal(as_string = True)
    volume = fields.Integer()
    
    @post_load
    def make_trade(self,data):
        return Trade(**data)
    
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string = True)
    high = fields.Decimal(as_string = True)
    low = fields.Decimal(as_string = True)
    close = fields.Decimal(as_string = True)
    volume = fields.Integer()
    
    @post_load
    def make_stock(self,data):
        data['open_']=data.pop('open')
        return Stock(**data)

class ActivitySchema(Schema):
    quotes = fields.Nested(StockSchema,many = True)
    trades = fields.Nested(TradeSchema,many = True)    

In [45]:
result = ActivitySchema().dumps(activity,indent=2)
print(result)

{
  "quotes": [
    {
      "high": "338.64",
      "date": "2018-11-22",
      "close": "338.19",
      "low": "337.60",
      "open": "338.19",
      "volume": 365607,
      "symbol": "TSLA"
    },
    {
      "high": "177.25",
      "date": "2018-11-22",
      "close": "176.78",
      "low": "176.64",
      "open": "176.66",
      "volume": 3699184,
      "symbol": "AAPL"
    },
    {
      "high": "103.48",
      "date": "2018-11-22",
      "close": "103.11",
      "low": "103.07",
      "open": "103.25",
      "volume": 4493689,
      "symbol": "MSFT"
    }
  ],
  "trades": [
    {
      "price": "338.25",
      "order": "buy",
      "volume": 100,
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "symbol": "TSLA"
    },
    {
      "price": "177.01",
      "order": "sell",
      "volume": 20,
      "timestamp": "2018-11-22T10:30:05",
      "commission": "9.99",
      "symbol": "AAPL"
    }
  ]
}


In [46]:
activity_dser = ActivitySchema().loads(result)
pprint(activity_dser)

TypeError: make_stock() got an unexpected keyword argument 'many'